In [37]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [38]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [ ]:
# Hyper-parameters 
input_size = 3*32*32
hidden_size = 500
num_classes = 10
num_epochs = 2
batch_size = 100
learning_rate = 0.0001

In [40]:
#dataset 
train_dataset = torchvision.datasets.CIFAR10(root='/home/fogman/my-venv/data', 
                                           train=True, 
                                           transform=transforms.ToTensor(),  
                                           download=True)
test_dataset = torchvision.datasets.CIFAR10(root='/home/fogman/my-venv/data', 
                                          train=False, 
                                          transform=transforms.ToTensor(),
                                          download=True)
print(train_dataset[2])

(tensor([[[1.0000, 0.9922, 0.9922,  ..., 0.9922, 0.9922, 0.9922],
         [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
         [1.0000, 0.9961, 0.9961,  ..., 0.9961, 0.9961, 0.9961],
         ...,
         [0.4431, 0.4353, 0.4118,  ..., 0.2824, 0.2824, 0.2824],
         [0.4353, 0.4078, 0.3882,  ..., 0.2667, 0.2745, 0.3059],
         [0.4157, 0.3882, 0.3725,  ..., 0.3059, 0.3098, 0.3137]],

        [[1.0000, 0.9922, 0.9922,  ..., 0.9922, 0.9922, 0.9922],
         [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
         [1.0000, 0.9961, 0.9961,  ..., 0.9961, 0.9961, 0.9961],
         ...,
         [0.4706, 0.4627, 0.4392,  ..., 0.3176, 0.3137, 0.3137],
         [0.4627, 0.4353, 0.4157,  ..., 0.2941, 0.2980, 0.3294],
         [0.4431, 0.4157, 0.4000,  ..., 0.3333, 0.3333, 0.3373]],

        [[1.0000, 0.9922, 0.9922,  ..., 0.9922, 0.9922, 0.9922],
         [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
         [1.0000, 0.9961, 0.9961,  ..., 0.9961, 0.9961, 0

In [41]:
# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

In [42]:
# Fully connected neural network with one hidden layer
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size) 
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)  
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

model = NeuralNet(input_size, hidden_size, num_classes).to(device)



print("Model's state_dict:")
for param_tensor in model.state_dict():
    print(param_tensor, "\t", model.state_dict()[param_tensor].size())

Model's state_dict:
fc1.weight 	 torch.Size([500, 3072])
fc1.bias 	 torch.Size([500])
fc2.weight 	 torch.Size([10, 500])
fc2.bias 	 torch.Size([10])


In [43]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)  

In [44]:
image, label = train_dataset[0]
image = image.reshape(-1, input_size).to(device)

label = torch.tensor([label]).to(device)

print(image)
print(label)


out = model(image)

loss = criterion(out,label)


print(out)
print(loss)

tensor([[0.2314, 0.1686, 0.1961,  ..., 0.5490, 0.3294, 0.2824]],
       device='cuda:0')
tensor([6], device='cuda:0')
tensor([[ 0.0092,  0.0233, -0.0048,  0.1283,  0.0305, -0.0173,  0.0200,  0.0710,
          0.0995, -0.0120]], device='cuda:0', grad_fn=<AddmmBackward0>)
tensor(2.3185, device='cuda:0', grad_fn=<NllLossBackward0>)


In [45]:
# Train the model
total_step = len(train_loader)
#print (total_step)

for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):  
        # Move tensors to the configured device
        images = images.reshape(-1,input_size).to(device)
        labels = labels.to(device)
        
        outputs = model(images)
        loss = criterion(outputs, labels)

        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/2], Step [100/500], Loss: 2.0032
Epoch [1/2], Step [200/500], Loss: 1.8313
Epoch [1/2], Step [300/500], Loss: 1.8477
Epoch [1/2], Step [400/500], Loss: 1.7057
Epoch [1/2], Step [500/500], Loss: 1.7349
Epoch [2/2], Step [100/500], Loss: 1.6907
Epoch [2/2], Step [200/500], Loss: 1.7010
Epoch [2/2], Step [300/500], Loss: 1.7756
Epoch [2/2], Step [400/500], Loss: 1.9620
Epoch [2/2], Step [500/500], Loss: 1.7408


In [46]:
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, input_size).to(device)
        labels = labels.to(device)
        
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total))

Accuracy of the network on the 10000 test images: 40.2 %


In [47]:
# Save the model checkpoint
torch.save(model.state_dict(), 'model1.ckpt')